# Necessary Common Functions

Those functions should be ran before each part.

In [2]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
from collections import Counter, defaultdict
import numba 
from numba import jit
import datasets
from datasets import Dataset as TFDataset
import multiprocessing

/root/autodl-tmp/xiaolong/.conda/envs/torch12/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def get_sessions(df: pd.DataFrame, test=False, list_item=False) -> list:
    
    all_item = []
    if 'next_item' in df and not test:
        if list_item:
            for i in trange(len(df)):
                all_item.append(np.concatenate([np.array(df.loc[i, 'prev_items']), np.array(df.loc[i, 'next_item'])], axis=0))
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items'][:-1]+f" '{df.loc[i, 'next_item']}']").replace(" ", ",")))
    else:
        if list_item:
            all_item = df['prev_items']
        else:
            for i in trange(len(df)):
                all_item.append(eval((df.loc[i, 'prev_items']).replace(" ", ",")))
    return all_item

In [4]:
def get_co_occurence_dict(sessions: list, bidirection: bool=True, weighted: bool=False, max_dis=None) -> dict:
    res = {}
    for sess in tqdm(sessions):
        for i, id in enumerate(sess):
            if id not in res:
                res[id] = Counter()
            
            if max_dis == None:
                e = len(sess)
            else:
                e = min(i + max_dis + 1, len(sess))

            for j in range(i+1, e):
                if not weighted:
                    res[id][sess[j]] += 1
                else:
                    res[id][sess[j]] += 1 / (j-i)
                if bidirection:
                    if sess[j] not in res:
                        res[sess[j]] = Counter()
                    if not weighted:
                        res[sess[j]][id] += 1
                    else:
                        res[sess[j]][id] += 1 / (j-i)
    return res

In [5]:
def sort_co_occurence_dict(co_occurence_dict: dict) -> dict:
    res = {}
    for k,v in co_occurence_dict.items():
        res[k] = dict(sorted(v.items(), key=lambda item: -item[1]))
    return res

In [6]:
def cast_dtype(df : pd.DataFrame, columns=None):
    if columns is None:
        columns = df.columns
    for k in columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [7]:
def get_session_last_item(session_df):
    last_items = []
    num_sessions = len(session_df)
    for i in tqdm(range(num_sessions)):
        sess = session_df.iloc[i]
        sess_prev_items = sess['prev_items']
        
        product_list = sess_prev_items.strip('[]').split(' ')
        last_item = product_list[-1].strip("'\n")

        last_items.append(last_item)
    return last_items 

In [8]:
def get_co_graph_counts(session_last_items, merged_candidates_df, co_graph_dict):
    co_graph_count_list = []
    for idx, row in tqdm(merged_candidates_df.iterrows(), total=merged_candidates_df.shape[0]):
        sess_id = row['sess_id']
        product = row['product']
        last_item = session_last_items[sess_id]
        co_graph_count = co_graph_dict[last_item][product]
        co_graph_count_list.append(co_graph_count)
    return co_graph_count_list

In [9]:
def flatten_co_graph_dict(co_graph_dict):
    product_list = []
    neighbor_list = []
    counts_list = []
    for product in tqdm(co_graph_dict.keys(), total=len(co_graph_dict)):
        for neigh in co_graph_dict[product].keys():
            product_list.append(product)
            neighbor_list.append(neigh)
            counts_list.append(co_graph_dict[product][neigh])
    return pd.DataFrame({'product_' : product_list, 'neighbor' : neighbor_list, 'counts' : counts_list})

In [10]:
def normalize_co_graph_counts(merged_candidates_counts):
    # normalize co graph counts 
    # merged_candidates_counts_g = cudf.from_pandas(merged_candidates_counts)
    sessions_count_sum = merged_candidates_counts[['sess_id', 'counts']].groupby('sess_id').sum()
    sessions_count_sum.sort_index(inplace=True)

    sessions_count_sum = sessions_count_sum.to_pandas()

    candidates_count_sum = sessions_count_sum.loc[merged_candidates_counts['sess_id']].reset_index(drop=True)
    merged_candidates_counts['counts_cum'] = candidates_count_sum['counts']
    merged_candidates_counts['normalized_counts'] = merged_candidates_counts['counts'] / merged_candidates_counts['counts_cum']

    # del merged_candidates_counts_g
    # del sessions_count_sum_g
    

# Merge valid co-graph counts 

In [11]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_2_feature.parquet'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'
product_data_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/products_train.csv'

In [12]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path)

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(product_data_path)

@lru_cache(maxsize=1)
def read_train_data():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_test_data():
    return pd.read_csv(test_sessions_path)

In [13]:
merged_candidates_feature = read_merged_candidates_feature()

In [14]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [15]:
train_sess_data = read_train_data()
valid_sess_data = read_valid_data()
test_sess_data = read_test_data()
product = read_product_data()

In [16]:
train_sess_item = get_sessions(train_sess_data, list_item=False)
valid_sess_item = get_sessions(valid_sess_data, test=True, list_item=False)

100%|██████████| 361581/361581 [01:03<00:00, 5739.31it/s] 


## bidirection

In [17]:
# bidirection
co_occurence_dict_bi = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=False)

100%|██████████| 3919479/3919479 [09:18<00:00, 7023.23it/s] 


In [18]:
# only one arg, can't use another arg
def get_bi_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_bi and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_bi[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 

    return cand_counter

In [19]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [20]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
bi_valid_sessions_counter = pool.map(get_bi_valid_session_co_graph_candidates, sess_ids)

In [21]:
valid_co_graph_candidates, len(bi_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [22]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(bi_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list

100%|██████████| 77570153/77570153 [28:55<00:00, 44689.19it/s]  
/tmp/ipykernel_246790/4091948514.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_0'] = all_items_co_graph_count_list


In [23]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_0'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']

/tmp/ipykernel_246790/860000090.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_246790/860000090.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0'] / merged_candidates['normalized_all_items_co_graph_count_0']


In [24]:
merged_candidates_feature['normalized_all_items_co_graph_count_0'] = merged_candidates['normalized_all_items_co_graph_count_0']
merged_candidates_feature['all_items_co_graph_count_0'] = merged_candidates['all_items_co_graph_count_0']

In [25]:
merged_candidates_feature.query('sess_id==20000').sort_values(by=['sasrec_scores_2'], ascending=False)[['product', 'sasrec_scores_2', 'all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0']][:25]

,product,sasrec_scores_2,all_items_co_graph_count_0,normalized_all_items_co_graph_count_0
4284904,B07D98ZNJX,25.679920,11,0.239130
4284905,B07D997JWG,24.778963,10,0.217391
4284854,B007SQ3HGS,19.812937,2,0.043478
4284918,B07QJB4MFD,18.794708,4,0.086957
4284942,B081N8XPQH,17.467945,0,0.000000
4285001,B09Y5GC5R7,17.269825,2,0.043478
4284966,B08SVPH33Z,16.826963,3,0.065217
4284939,B081N7R51C,16.567038,1,0.021739
4284927,B07V4PBMWT,15.528075,2,0.043478
4284931,B07X9Q39H6,14.468607,2,0.043478


## uni and weighted

In [26]:
# weight 
# co_occurence_dict_bi_weight = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=True)
co_occurence_dict_uni_weight = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=False, weighted=True)

100%|██████████| 3919479/3919479 [06:52<00:00, 9505.30it/s] 


In [27]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [28]:
# only one arg, can't use another arg
def get_uni_wei_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_weight and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_weight[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [29]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [30]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
uni_wei_valid_sessions_counter = pool.map(get_uni_wei_valid_session_co_graph_candidates, sess_ids)

In [31]:
valid_co_graph_candidates, len(uni_wei_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [32]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(uni_wei_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_1'] = all_items_co_graph_count_list

100%|██████████| 77570153/77570153 [19:31<00:00, 66207.15it/s]  
/tmp/ipykernel_246790/1702843849.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_1'] = all_items_co_graph_count_list


In [33]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_1'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1'] / merged_candidates['normalized_all_items_co_graph_count_1']

/tmp/ipykernel_246790/1383280374.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_1'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_246790/1383280374.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1'] / merged_candidates['normalized_all_items_co_graph_count_1']


In [34]:
merged_candidates_feature['normalized_all_items_co_graph_count_1'] = merged_candidates['normalized_all_items_co_graph_count_1']
merged_candidates_feature['all_items_co_graph_count_1'] = merged_candidates['all_items_co_graph_count_1']

In [35]:
merged_candidates_feature.query('sess_id==10000').sort_values(by=['sasrec_scores_2'], ascending=False)[['product', 'sasrec_scores_2', 'all_items_co_graph_count_1', 'normalized_all_items_co_graph_count_1']][:25]

,product,sasrec_scores_2,all_items_co_graph_count_1,normalized_all_items_co_graph_count_1
2141926,B07SC4G9ZH,20.979120,2.000000,0.013207
2142045,B09Q68WNR3,14.378578,7.416667,0.048974
2142072,B0B61SRMWJ,13.956560,1.000000,0.006603
2141935,B07W6HZJ53,13.931480,0.000000,0.000000
2142018,B09JGND6WB,13.371040,1.690909,0.011166
2142040,B09NX3RSV8,13.268908,2.000000,0.013207
2141934,B07W6HXHN7,12.769808,0.000000,0.000000
2141840,B00K8OOYXW,12.667303,0.000000,0.000000
2141927,B07SFR9T9V,12.622787,4.000000,0.026413
2142050,B09SZG3766,12.610863,0.000000,0.000000


## uni and dis=1

In [36]:
# max dis = 1 
# co_occurence_dict_bi_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=True, weighted=False, max_dis=1)
co_occurence_dict_uni_dis1 = get_co_occurence_dict(train_sess_item + valid_sess_item, bidirection=False, weighted=False, max_dis=1)

100%|██████████| 3919479/3919479 [04:11<00:00, 15590.37it/s]


In [37]:
merged_candidates = merged_candidates_feature[['sess_id', 'sess_locale', 'product']]

In [38]:
# only one arg, can't use another arg
def get_uni_dis1_valid_session_co_graph_candidates(sess_id):
    sess = valid_sess_item[sess_id]
    prev_items = set()
    cand_counter = Counter()
    for item in sess:
        if item in co_occurence_dict_uni_dis1 and item not in prev_items:
            cand_counter = cand_counter + co_occurence_dict_uni_dis1[item]
            prev_items.add(item) # one time for every item
    for item in sess:
        if item in cand_counter:
            cand_counter.pop(item) # remove history items 
    
    return cand_counter

In [39]:
valid_co_graph_candidates = TFDataset.from_dict({'sess_id' : list(range(valid_sess_data.shape[0]))})

In [40]:
# about 1 mins
sess_ids = list(range(len(valid_sess_item)))
pool = multiprocessing.Pool(10)
uni_dis1_valid_sessions_counter = pool.map(get_uni_dis1_valid_session_co_graph_candidates, sess_ids)

In [41]:
valid_co_graph_candidates, len(uni_dis1_valid_sessions_counter)

(Dataset({
     features: ['sess_id'],
     num_rows: 361581
 }),
 361581)

In [42]:
all_items_co_graph_count_list = []
for row in tqdm(merged_candidates.itertuples(), total=merged_candidates.shape[0]):
    all_items_co_graph_count_list.append(uni_dis1_valid_sessions_counter[row.sess_id][row.product])
assert len(all_items_co_graph_count_list) == merged_candidates.shape[0]
merged_candidates['all_items_co_graph_count_2'] = all_items_co_graph_count_list

100%|██████████| 77570153/77570153 [06:59<00:00, 185022.63it/s]
/tmp/ipykernel_246790/3640618043.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['all_items_co_graph_count_2'] = all_items_co_graph_count_list


In [43]:
count_sum_array = merged_candidates.groupby(by='sess_id')['all_items_co_graph_count_2'].sum().to_numpy()
assert len(count_sum_array[merged_candidates['sess_id']]) == merged_candidates.shape[0]
merged_candidates['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates['sess_id']]
merged_candidates['normalized_all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2'] / merged_candidates['normalized_all_items_co_graph_count_2']

/tmp/ipykernel_246790/2314243688.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_2'] = count_sum_array[merged_candidates['sess_id']]
/tmp/ipykernel_246790/2314243688.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_candidates['normalized_all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2'] / merged_candidates['normalized_all_items_co_graph_count_2']


In [44]:
merged_candidates_feature['normalized_all_items_co_graph_count_2'] = merged_candidates['normalized_all_items_co_graph_count_2']
merged_candidates_feature['all_items_co_graph_count_2'] = merged_candidates['all_items_co_graph_count_2']

In [45]:
merged_candidates_feature.query('sess_id==150000').sort_values(by=['sasrec_scores_2'], ascending=False)[['product', 'sasrec_scores_2', 'all_items_co_graph_count_2', 'normalized_all_items_co_graph_count_2']][:25]

,product,sasrec_scores_2,all_items_co_graph_count_2,normalized_all_items_co_graph_count_2
32166952,B09B31TB29,21.632847,3,0.500000
32166966,B09FZCB5P6,15.956667,1,0.166667
32166995,B0B6G1NYY3,14.619350,0,0.000000
32166871,B075569D3Z,12.117474,0,0.000000
32166994,B0B6FT1VPK,11.932799,0,0.000000
32166887,B07NS3JSDM,11.876396,0,0.000000
32166961,B09FKW2TQR,11.817984,0,0.000000
32166964,B09FKXH2QG,11.722295,0,0.000000
32166836,B00M4SN1DE,10.663493,0,0.000000
32166884,B07NRLQ5MH,10.564134,0,0.000000


In [46]:
cast_dtype(merged_candidates_feature, 
           ['all_items_co_graph_count_0', 'normalized_all_items_co_graph_count_0', 'all_items_co_graph_count_1', 'normalized_all_items_co_graph_count_1', 'all_items_co_graph_count_2', 'normalized_all_items_co_graph_count_2'])
merged_candidates_feature.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [47]:
merged_candidates_feature

,sess_id,sess_locale,product,target,sess_avg_price,product_price,product_freq,sasrec_scores_2,normalized_sasrec_scores_2,sasrec_scores_3,...,gru4rec_scores,normalized_gru4rec_scores,gru4rec_scores_2,normalized_gru4rec_scores_2,normalized_all_items_co_graph_count_0,all_items_co_graph_count_0,normalized_all_items_co_graph_count_1,all_items_co_graph_count_1,normalized_all_items_co_graph_count_2,all_items_co_graph_count_2
0,0,UK,B000V599Y2,0.0,7.388571,5.200000,37.0,13.152878,7.433639e-04,10.677187,...,4.342063,4.265118e-06,12.142086,4.663249e-05,0.003432,3,0.000000,0.000000,0.000000,0
1,0,UK,B007VZUA7U,0.0,7.388571,7.000000,36.0,9.393598,1.732076e-05,8.838863,...,2.631823,7.712291e-07,11.735471,3.105259e-05,0.003432,3,0.000000,0.000000,0.000000,0
2,0,UK,B009EUAEQC,0.0,7.388571,7.490000,4.0,11.754339,1.835794e-04,10.670128,...,9.855083,1.057317e-03,12.416017,6.132748e-05,0.006865,6,0.003793,1.033333,0.000000,0
3,0,UK,B00AH02IWG,0.0,7.388571,8.500000,3.0,12.194766,2.851667e-04,11.166204,...,8.191824,2.003831e-04,11.401733,2.224116e-05,0.004577,4,0.004588,1.250000,0.006494,1
4,0,UK,B00I0UKKD4,0.0,7.388571,17.049999,118.0,11.835367,1.990737e-04,11.681271,...,13.495910,4.030846e-02,16.872612,5.286033e-03,0.003432,3,0.006730,1.833333,0.006494,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77570148,361580,DE,B0BB7XV97M,0.0,32.424000,47.990002,56.0,9.117821,6.076918e-05,9.635838,...,9.268379,1.396744e-05,14.038595,8.990213e-05,0.003350,2,0.000000,0.000000,0.000000,0
77570149,361580,DE,B0BB7YSRBX,0.0,32.424000,43.990002,58.0,9.163816,6.362959e-05,9.159988,...,7.047796,1.516107e-06,13.342258,4.480792e-05,0.001675,1,0.000000,0.000000,0.000000,0
77570150,361580,DE,B0BB7ZMGY8,0.0,32.424000,41.990002,452.0,11.256460,5.158017e-04,10.119755,...,9.359167,1.529486e-05,12.778135,2.548937e-05,0.038526,23,0.010612,1.226190,0.000000,0
77570151,361580,DE,B0BD4CP7N3,0.0,32.424000,24.990000,1.0,-3.778687,1.523355e-10,-1.612869,...,-0.593306,7.281843e-10,-3.986487,1.335292e-12,0.000000,0,0.000000,0.000000,0.000000,0
